## Práctica 3. Optimización con algoritmos genéticos

Para los modelos de aprendizaje automático vamos a utilizar la librería Scikit-learn (o sklearn), una biblioteca de machine learning para Python que proporciona herramientas simples y eficientes para el análisis y modelado de datos. Es ampliamente utilizada en la comunidad de ciencia de datos y aprendizaje automático debido a su facilidad de uso y la robustez de sus algoritmos.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine, fetch_california_housing
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import ParameterGrid, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import time
from tqdm import tqdm
import random

import warnings
import sys
import os
from itertools import product
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses


## Carga de datasets
Puedes elegir cualquiera de los datasets de prueba que vienen con la librería sklearn 
https://scikit-learn.org/1.5/datasets/toy_dataset.html

O utilizar tu propio archivo de datos en formato csv. Vamos a usar el dataset de titanic que te puedes descargar de Kaggle.

Después los dividimos en dos partes, conjunto de entrenamiento y conjunto de prueba (para evaluar el modelo entrenado)

In [2]:
# Cargar dataset wine que viene como ejemplo en sklearn: 
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine
dataset = load_wine()
X = dataset.data
y = dataset.target
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [3]:
# Cargar dataset de precios de casas que viene como ejemplo en sklearn (se binariza el target): 
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing
dataset = fetch_california_housing()
dataset.target = np.where(dataset.target > np.median(dataset.target), 1, 0)
X = dataset.data
y = dataset.target
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
# Cargar el dataset de Titanic, que se puede descargar de Kaggle:
#https://www.kaggle.com/c/titanic/data
dataset = pd.read_csv('titanic.csv')
X = dataset[dataset.columns[:-1]]
y = dataset[dataset.columns[-1:]]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelos que vamos a optimizar 

Puedes elegir cualquier de los dos modelos siguientes para optimizar. 

## Modelo con XGBoost

XGBoost es una biblioteca optimizada y eficiente de machine learning basada en el algoritmo de Gradient Boosting. Su nombre es un acrónimo de "Extreme Gradient Boosting". XGBoost es conocido por su alto rendimiento y velocidad.
XGBoost se basa en el principio de Gradient Boosting, donde se construyen modelos secuencialmente, y cada nuevo modelo trata de corregir los errores cometidos por el modelo anterior. Específicamente en cada iteración, se ajusta un modelo débil (usualmente un árbol de decisión) a los residuos (errores) de los modelos anteriores. El objetivo es minimizar una función de pérdida usando el gradiente descendente, lo que le permite mejorar iterativamente la precisión del modelo.

En esta práctica vamos a elegir los parámetros que optimizan la precisión del modelo usando un algoritmo genético. 

In [11]:
hyperparameter_space = {
    "gamma": np.arange(0, 0.5, 0.1),
    "learning_rate": np.array([0.01, 0.1, 0.2, 0.3, 0.4]),
    "max_depth": np.arange(3, 10, 1), 
    "n_estimators": np.arange(50, 750, 50), 
    "subsample": np.arange(0.4, 1.0, 0.1),
    "min_child_weight" : np.arange(0, 10, 1),
    "subsample": np.arange(0.4, 1.0, 0.1),
    "colsample_bytree": np.arange(0.4, 1.0, 0.1),
    "scale_pos_weight": np.arange(0, 50, 10),
}

# ParameterGrid crea un iterable de todas las combinaciones posibles de hiperparámetros dados en forma de diccionario.
# ParameterGrid se usa a menudo dentro de GridSearchCV

search_space = list(ParameterGrid(hyperparameter_space))
print('Combinations: ', len(search_space))

# Creamos el modelo. Todavía no está entrenado
xgb_model = xgb.XGBClassifier( n_jobs=-1, random_state=42)
model = xgb_model

# Entrenar el modelo
model.fit(X_train, y_train)


Combinations:  4410000


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

Esto sería un ejemplo de cómo crear un modelo XGBClassifier con algunos hiperparámetros personalizados
¿Cómo sabemos cuáles son los mejores valores? 
Se prueban y se evaluan modelos con distintas configuraciones de hiperparámetros. Hay 4410000 combinaciones para elegir. 

    model = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        min_child_weight=1,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0,
        reg_alpha=0.01,
        reg_lambda=1,
        scale_pos_weight=1,
        objective='binary:logistic',
        n_jobs=-1,
        random_state=42
    )


## Modelo con Perceptron Multicapa
Este modelo es considerablemente más lento que XGBoost porque no está paralelizado

In [6]:
first_layer_neurons = np.arange(10, 100, 10)
second_layer_neurons = np.arange(10, 100, 10)
hidden_layer_sizes = list(product(first_layer_neurons, second_layer_neurons))

hyperparameter_space = {
    "hidden_layer_sizes": hidden_layer_sizes,
    "activation": ['logistic', 'tanh', 'relu'],
    "solver": ['sgd', 'adam', 'lbfgs'],
    "alpha": 10.0 ** -np.arange(1, 10),
    "learning_rate": ['constant', 'adaptive']
}


search_space = list(ParameterGrid(hyperparameter_space))
print('Combinations: ', len(search_space))

# MLP is more "delicate" than XGBoost, so we need to preprocess the data removing NaNs and scaling it
imp = SimpleImputer( strategy='mean')
X = imp.fit_transform(X)
scaler = StandardScaler()
X = scaler.fit_transform(X)

mlp_model = MLPClassifier(max_iter = 1000, random_state=42)
model = mlp_model

Combinations:  13122


# Busqueda de los mejores hiperparametros con validación cruzada utilizando los métodos de scikit-learn

Cuando ajustamos un modelo de machine learning, los hiperparámetros (como la profundidad máxima de un árbol, la tasa de aprendizaje, el número de estimadores, etc.) juegan un papel crucial en el rendimiento del modelo. Seleccionar los valores óptimos de estos hiperparámetros es importante para mejorar la precisión y evitar el sobreajuste.
La librería Scikit-learn ofrece sus propios métodos de optimización de hiperparámetros, por ejemplo, en vez de recorrer exhaustivamente todas las combinaciones posibles (como hace GridSearchCV), una opción mejor es usar RandomizedSearchCV que explora un número aleatorio de combinaciones de hiperparámetros. RandomizedSearchCV selecciona aleatoriamente un número especificado de combinaciones y evalúa el rendimiento del modelo para cada una de ellas. Esto reduce significativamente el tiempo de búsqueda, haciendo posible encontrar buenos (aunque no necesariamente óptimos) conjuntos de hiperparámetros con mayor rapidez. 

Aun así puedes comprobar que tarda bastante. Horas (dependiendo del dataset y del modelo). GridSearch es más ineficiente. 

In [7]:
start_time = time.time()
search = RandomizedSearchCV(model, param_distributions=hyperparameter_space, random_state=42, n_iter=200, cv=5, verbose=1, n_jobs=-1, return_train_score=True)
search.fit(X_train, y_train)
print("Time: %s seconds " % (time.time() - start_time))
print(search.best_score_)
print(search.best_params_)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


KeyboardInterrupt: 

In [8]:
start_time = time.time()
search = GridSearchCV(model, param_grid=hyperparameter_space, cv=5, verbose=2, n_jobs=-1, return_train_score=True)
search.fit(X_train, y_train)
print("Time: %s seconds " % (time.time() - start_time))
print(search.best_score_)
print(search.best_params_)

Fitting 5 folds for each of 13122 candidates, totalling 65610 fits


KeyboardInterrupt: 

# Opciones de búsqueda exhaustiva.
    1: Búsqueda greedy: tarda varios días en ejecutarse (con suerte)

In [13]:
hyperparameter_space = {
    "gamma": np.arange(0, 0.5, 0.1),
    "learning_rate": np.array([0.01, 0.1, 0.2, 0.3, 0.4]),
    "max_depth": np.arange(3, 10, 1), 
    "n_estimators": np.arange(50, 750, 50), 
    "subsample": np.arange(0.4, 1.0, 0.1),
    "min_child_weight" : np.arange(0, 10, 1),
    "subsample": np.arange(0.4, 1.0, 0.1),
    "colsample_bytree": np.arange(0.4, 1.0, 0.1),
    "scale_pos_weight": np.arange(0, 50, 10),
}

# ParameterGrid crea un iterable de todas las combinaciones posibles de hiperparámetros dados en forma de diccionario.
# ParameterGrid se usa a menudo dentro de GridSearchCV

search_space = list(ParameterGrid(hyperparameter_space))
print('Combinations: ', len(search_space))

# Creamos el modelo. Todavía no está entrenado
xgb_model = xgb.XGBClassifier( n_jobs=-1, random_state=42)
model = xgb_model

# Entrenar el modelo
model.fit(X_train, y_train)


Combinations:  4410000


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

    2: búsqueda random: tarda unas horas en ejecutarse dependiendo del dataset

# Práctica 3: Algortimo genético 

Se pide resolver el problema de optimizar los hiperparámetros de alguno de los modelos propuestos usando un algoritmo genético. 
Puedes implementar tus propias variaciones de las funciones de selección, cruce y mutación o utilizar las que hemos visto en la sesión anterior. 
Evalua y comenta los resultados obtenidos y comparalos con otros métodos de optimización de parámetros. 

In [ ]:
class GeneticSearch:
    def __init__(self, X, y, hyperparameter_space, model):
        self.X = X
        self.y = y
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.hyperparameter_space = hyperparameter_space
        self.population = []
        self.model = model
        
    # Initizaling the population randomly
    def initialize_population(self, population_size):
        self.population = [{key: random.choice(values) for key, values in self.hyperparameter_space.items()} for _ in range(population_size)]

    # Completing the population up to the desired size 
    def complete_population(self, population_size):
        extension_size = population_size - len(self.population)
        self.population.extend([{key: random.choice(values) for key, values in self.hyperparameter_space.items()} for _ in range(extension_size)])

    # Calculating the fitness scores of the population
    def calculate_fitness(self):
        self.evaluation = []
        for params in self.population:
            model = self.model.set_params(**params)
            try: #Some combinations of hyperparameters may not be valid and throw an error
                model.fit(self.X_train, self.y_train)
                # Make predictions on the testing data
                y_pred = model.predict(self.X_test)
                # Evaluate the model
                score = accuracy_score(self.y_test, y_pred)
            except:
                score = 0          
            self.evaluation.append((score, params))
        self.evaluation = sorted(self.evaluation, key=lambda x: x[0], reverse=True)
    
    # Selecting the best performing individuals
    def select_best(self, percentage, shuffle=False):
        k=round(len(self.evaluation) * percentage)
        best = [x[1] for x in self.evaluation[:k]]
        if shuffle:
            random.shuffle(best)
        return best

    # Crossing over the parent choromosomes
    def crossover(self, parent1, parent2):
        # Generating a random crossover point
        crossover_point = np.random.randint(1, len(parent1))
        # Altering the parent choromosomes
        child1 = dict(list(parent1.items())[:crossover_point] + list(parent2.items())[crossover_point:])
        child2 = dict(list(parent2.items())[:crossover_point] + list(parent1.items())[crossover_point:])
        return child1, child2

    # Crossing over the parent choromosomes by taking the mean of the genes (only works for numeric parameters)
    def crossover_mean(self, parent1, parent2):
        child1 = parent1.copy()
        child2 = parent2.copy()
        for key in parent1.keys():
            if random.choice([True, False]):
                child1[key] = (parent1[key] + parent2[key]) / 2
            else:
                child2[key] = (parent1[key] + parent2[key]) / 2

            if parent1[key].dtype == 'int64':
                child1[key] = np.int64(child1[key])
                child2[key] = np.int64(child2[key])

        return child1, child2

    # Mutating the genes
    def mutate(self, individual, mutation_rate):
        # Randomly selecting mutating genes. Generates an array with random boolean values, with the same length as the individual
        mask = np.random.rand(len(individual)) < mutation_rate
        # Labeling the genes
        genes = [item for i, item in enumerate(individual.items()) if mask[i]]
        # Creating new variations for labeled genes by randomly selecting from the hyperparameter space
        new_genes = [(param, random.choice(self.hyperparameter_space[param])) for param, value in genes]
        for param, value in new_genes:
            individual[param] = value
        return individual

    # Performs the search
    def search(self, generations=100, population_size=50, elite_rate=.1, selection_rate=.6, mutation_rate=0.1):
        # Initializing the first population
        population = self.initialize_population(population_size)
        # Calculating the initial fitness scores 
        self.calculate_fitness()
        
        print(f"Initial population, Best Score: {self.evaluation[0][0]}, Best Config: {self.evaluation[0][1]}")

        # Looping through each generation
        pbar=tqdm(range(generations))
        for generation in pbar:        
            # keep the best performing choromosomes
            elite = self.select_best(elite_rate)

            # Select best individuals for mating (you may want to shuffle the selection or not)
            selection = self.select_best(selection_rate, shuffle=True)

            # New population initialization
            self.population = elite.copy()

            # Crossing over the parent choromosomes in pairs
            for i in range(int((len(selection) / 2))):
                parent1 = selection[2*i]
                parent2 = selection[2*i+1]
                # Crossing over the parent choromosomes
                # You can use either crossover or crossover_mean
                child1, child2 = self.crossover(parent1, parent2)
                # Mutating the genes
                child1 = self.mutate(child1, mutation_rate)
                child2 = self.mutate(child2, mutation_rate)
                # Adding new offsprings to population
                self.population.extend([child1, child2])


            # Completing the population
            self.complete_population(population_size)

            # Calculating the fitness scores 
            self.calculate_fitness()

            pbar.set_description(f"Generation: {generation+1}, Best Score: {self.evaluation[0][0]}")

        print(f"Best Score: {self.evaluation[0][0]}, Best Config: {self.evaluation[0][1]}")  
        return self.evaluation[0]


In [ ]:
genetic_search = GeneticSearch(X, y, hyperparameter_space, model)
genetic_search.search(generations=20, population_size=10, elite_rate=.1, selection_rate=.6, mutation_rate=0.2)
